In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

DATA_PATH = "../data"

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

from src.log_mock import PrintLog
log = PrintLog()

import wandb

wandb.init(mode="disabled")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [3]:
wapi = wandb.Api()
runs = wapi.runs("bayes/poverty")

In [4]:
for run in runs:
    print(run.name, run.summary.keys())

bbb-5-(E) dict_keys(['mse', 'wilds', 'id', 'sqce', '_wandb', 'avg_ll', '_timestamp', 'train_loss', 'ood', '_step', 'pearson', 'qce', 'avg_lml', '_runtime'])
bbb-5-(D) dict_keys(['ood', '_wandb', 'avg_lml', 'pearson', '_timestamp', 'wilds', 'train_loss', 'avg_ll', '_runtime', 'id', 'mse', 'qce', 'sqce', '_step'])
bbb-5-(C) dict_keys(['sqce', 'train_loss', 'mse', '_runtime', 'avg_lml', '_step', 'wilds', 'id', 'qce', '_wandb', 'avg_ll', 'pearson', '_timestamp', 'ood'])
bbb-5-(B) dict_keys(['_step', 'train_loss', 'mse', 'ood', 'avg_ll', 'avg_lml', '_runtime', 'id', '_timestamp', 'sqce', 'wilds', '_wandb', 'pearson', 'qce'])
bbb-5-(A) dict_keys(['_step', '_timestamp', 'sqce', 'avg_lml', 'mse', 'ood', 'wilds', 'pearson', '_wandb', 'avg_ll', '_runtime', 'train_loss', 'id', 'qce'])
ivon_p100-1-(A) dict_keys(['sqce', '_wandb', 'avg_lml', 'train_loss', 'mse', 'pearson', 'ood', 'qce', 'wilds', 'avg_ll', '_runtime', 'id', '_timestamp', '_step'])
ivon_p1-1-(A) dict_keys(['_wandb'])
ivon_p10-1-(A) d

In [5]:
import plotly.express as px
import pandas as pd
import dateutil
import datetime

def create_plot_data_for_run(run):
    model_name = run.name.split("-")[0]
    return {
        "model": model_name + "-" + run.name.split("-")[1],
        "ood mse": run.summary["ood"]["mse"],
        "ood pearson": run.summary["ood"]["pearson"],
        "ood ll": run.summary["ood"]["avg_ll"],
        "ood lml": run.summary["ood"]["avg_lml"],
        "ood qce": run.summary["ood"]["qce"],
        "ood sqce": run.summary["ood"]["sqce"],
        "id mse": run.summary["id"]["mse"],
        "id pearson": run.summary["id"]["pearson"],
        "id ll": run.summary["id"]["avg_ll"],
        "id lml": run.summary["id"]["avg_lml"],
        "id qce": run.summary["id"]["qce"],
        "id sqce": run.summary["id"]["sqce"],
    }

def plot(data, value):
    plot = px.box(data, x="model", y=value, color="model")
    return plot

def pareto_plot(data, x, y):
    plot = px.scatter(data, x=x, error_x=f"{x}_std", y=y, error_y=f"{y}_std", color="model")
    return plot

def build_data(runs):
    rows = []
    for run in runs:
        if run.state != "finished":
            continue
        if "old" in run.tags:
            print("Skipping old run " + run.name)
            continue
        if "ood" not in run.summary:
            print("Skipping failed run " + run.name)
            continue
        rows.append(create_plot_data_for_run(run))
    return pd.DataFrame.from_dict(rows)

def aggregate_data(data):
    aggregated_data = data.groupby(["model"]).agg({
        "model": "first",
        "ood mse": ["mean", "sem"], 
        "ood pearson": ["mean", "sem"], 
        "ood ll": ["mean", "sem"], 
        "ood lml": ["mean", "sem"],
        "ood qce": ["mean", "sem"],
        "ood sqce": ["mean", "sem"],
        "id mse": ["mean", "sem"], 
        "id pearson": ["mean", "sem"], 
        "id ll": ["mean", "sem"], 
        "id lml": ["mean", "sem"],
        "id qce": ["mean", "sem"],
        "id sqce": ["mean", "sem"],
    })
    aggregated_data.columns = [a[0] + "_std" if a[1] == "sem" else a[0] for a in aggregated_data.columns.to_flat_index()]
    aggregated_data["ood mse_std"] *= 2.0
    aggregated_data["ood pearson_std"] *= 2.0
    aggregated_data["ood ll_std"] *= 2.0
    aggregated_data["ood lml_std"] *= 2.0
    aggregated_data["ood qce_std"] *= 2.0
    aggregated_data["ood sqce_std"] *= 2.0
    aggregated_data["id mse_std"] *= 2.0
    aggregated_data["id pearson_std"] *= 2.0
    aggregated_data["id ll_std"] *= 2.0
    aggregated_data["id lml_std"] *= 2.0
    aggregated_data["id qce_std"] *= 2.0
    aggregated_data["id sqce_std"] *= 2.0
    return aggregated_data

In [6]:
data = aggregate_data(build_data(runs))

Skipping failed run ivon_p1-1-(A)
Skipping failed run ivon_p10-1-(A)
Skipping failed run bbb-5-(E)
Skipping failed run bbb-5-(D)
Skipping failed run bbb-5-(C)
Skipping failed run bbb-5-(B)
Skipping old run swag-1-(E)
Skipping old run swag-1-(D)
Skipping old run swag-1-(C)
Skipping old run swag-1-(B)
Skipping old run swag-1-(A)
Skipping old run map-1-(0)
Skipping old run map-1-(0)
Skipping old run map-1-(0)
Skipping old run map-1-(0)
Skipping old run map-1-(0)


In [7]:
data

,model,ood mse,ood mse_std,ood pearson,ood pearson_std,ood ll,ood ll_std,ood lml,ood lml_std,ood qce,...,id pearson,id pearson_std,id ll,id ll_std,id lml,id lml_std,id qce,id qce_std,id sqce,id sqce_std
model,,,,,,,,,,,,,,,,,,,,,
bbb-1,bbb-1,0.264414,0.054177,0.499581,0.071628,-7.881385,2.289589,-12.074589,2.469800,0.333063,...,0.679819,0.019468,-4.507808,0.269432,-7.224426,0.754106,0.28620,0.014265,-0.28620,0.014265
bbb-5,bbb-5,0.251532,0.047954,0.517792,0.074439,-6.256722,1.461637,-11.498457,2.299434,0.309490,...,0.694331,0.013835,-3.619075,0.343549,-7.159320,0.585912,0.25596,0.012439,-0.25596,0.012439
ivon_p100-1,ivon_p100-1,0.358961,NaN,0.212930,NaN,-4.328319,NaN,-21.933134,NaN,0.212894,...,0.539511,NaN,-3.693469,NaN,-17.517288,NaN,0.16580,NaN,-0.16580,NaN
ivon_p500-1,ivon_p500-1,0.346865,NaN,0.249217,NaN,-4.657480,NaN,-19.787493,NaN,0.235705,...,0.570705,NaN,-3.887586,NaN,-15.832489,NaN,0.20100,NaN,-0.20100,NaN
laplace-1,laplace-1,0.279661,0.043824,0.473024,0.072380,-12.599394,2.191216,-12.599394,2.191216,0.386503,...,0.663531,0.018175,-7.823242,0.736995,-7.823242,0.736995,0.35688,0.006419,-0.35688,0.006419
laplace-5,laplace-5,0.250690,0.044219,0.515964,0.079857,-5.614497,1.271065,-12.323619,2.141291,0.264577,...,0.701742,0.022955,-3.233737,0.539446,-7.474223,0.828127,0.20598,0.004448,-0.20598,0.004448
map-1,map-1,0.266579,0.040839,0.486679,0.074388,-11.945303,2.041965,-11.945304,2.041965,0.382362,...,0.672504,0.019290,-7.445338,0.761398,-7.445338,0.761398,0.34772,0.008277,-0.34772,0.008277
map-5,map-5,0.249124,0.043270,0.520412,0.075467,-6.126161,1.421901,-12.113140,2.074442,0.283459,...,0.703414,0.021684,-3.438219,0.568580,-7.092774,0.744104,0.22808,0.010000,-0.22808,0.010000
mcd-1,mcd-1,0.258931,0.049335,0.491167,0.078773,-6.867588,1.719686,-12.175314,2.398080,0.315976,...,0.694990,0.010089,-3.603860,0.482835,-7.236589,0.673005,0.26696,0.013712,-0.26696,0.013712


In [8]:
pareto_plot(data, "ood pearson", "ood sqce")

In [9]:
pareto_plot(data, "id pearson", "id sqce")

In [10]:
pareto_plot(data, "ood lml", "ood ll")

In [11]:
data.to_csv(sep=",", header=True)

'model,model,ood mse,ood mse_std,ood pearson,ood pearson_std,ood ll,ood ll_std,ood lml,ood lml_std,ood qce,ood qce_std,ood sqce,ood sqce_std,id mse,id mse_std,id pearson,id pearson_std,id ll,id ll_std,id lml,id lml_std,id qce,id qce_std,id sqce,id sqce_std\nbbb-1,bbb-1,0.26441408395767213,0.054177201843491896,0.4995814644109208,0.07162765526619304,-7.881385374069214,2.289589118870818,-12.074589347839355,2.469799686319503,0.33306310772895814,0.03584951410287148,-0.33306310772895814,0.03584951410287148,0.16894285380840302,0.013531736044599648,0.679818681723485,0.019467699594020126,-4.5078083038330075,0.2694315314728759,-7.2244264602661135,0.754106256688152,0.28620001673698425,0.014264783267151193,-0.28620001673698425,0.014264783267151193\nbbb-5,bbb-5,0.2515317678451538,0.04795432241944033,0.5177920572981668,0.0744388574691839,-6.256721687316895,1.461636593417048,-11.498457431793213,2.2994338545696342,0.3094897150993347,0.027268437630692182,-0.3094897150993347,0.027268437630692182,0.16070

In [12]:
algo_names = [
    ("map-1", "MAP"),
    ("map-5", "Deep Ensemble"),
    ("mcd-1", "MCD"),
    ("mcd-5", "MultiMCD"),
    ("swag-1", "SWAG"),
    ("swag-5", "MultiSWAG"),
    ("laplace-1", "LL Laplace"),
    ("laplace-5", "LL MultiLaplace"),
    ("bbb-1", "BBB"),
    ("bbb-5", "MultiBBB"),
    ("rank1-1", "Rank-1 VI"),
    ("ivon_p500-1", "iVON"),
    ("svgd-1", "SVGD"),
]

def num(value, std, best=None, ty=None):
    value = float(value)
    std = float(std)

    if not math.isnan(std):
        num_string = f"{value:.3f} \\pm {std:.3f}"
    else:
        num_string = f"{value:.3f} \\pm -"

    if best is None or ty is None or math.isnan(std):
        return f"${num_string}$"

    if ty == "max":
        if value >= best:
            num_string = f"\\bm{{{num_string}}}"
    elif ty == "min":
        if value <= best:
            num_string = f"\\bm{{{num_string}}}"
    elif ty == "zero":
        if abs(value) <= best:
            num_string = f"\\bm{{{num_string}}}"
    return f"${num_string}$"

def col_name(name, align):
    return f"\\multicolumn{{1}}{{{align}}}{{{name}}}"

def create_table(data, prefix):
    print("\\begin{tabular}{l|rrrrrr}")
    print(f"    {col_name('Model', 'l')} & {col_name('Worst U/R Pearson', 'c')} & {col_name('psLML', 'c')} & {col_name('LML', 'c')} & {col_name('MSE', 'c')} & {col_name('QCE', 'c')} & {col_name('sQCE', 'c')} \\\\")
    print("    \\hline")

    best_pearson, best_pearson_std = 0, 0
    best_ll, best_ll_std = -1000, 0
    best_lml, best_lml_std = -1000, 0
    best_mse, best_mse_std = 1000, 0
    best_qce, best_qce_std = 1000, 0
    best_sqce, best_sqce_std = 1000, 0

    for algo, name in algo_names:
        row = data[data["model"] == algo]
        
        if float(row[prefix + "pearson"]) > best_pearson:
            best_pearson = float(row[prefix + "pearson"])
            best_pearson_std = float(row[prefix + "pearson_std"])

        if float(row[prefix + "ll"]) > best_ll:
            best_ll = float(row[prefix + "ll"])
            best_ll_std = float(row[prefix + "ll_std"])

        if float(row[prefix + "lml"]) > best_lml:
            best_lml = float(row[prefix + "lml"])
            best_lml_std = float(row[prefix + "lml_std"])

        if float(row[prefix + "mse"]) > best_mse:
            best_mse = float(row[prefix + "mse"])
            best_mse_std = float(row[prefix + "mse_std"])
        
        if float(row[prefix + "qce"]) < best_qce:
            best_qce = float(row[prefix + "qce"])
            best_qce_std = float(row[prefix + "qce_std"])
        
        if abs(float(row[prefix + "sqce"])) < best_sqce:
            best_sqce = abs(float(row[prefix + "sqce"]))
            best_sqce_std = float(row[prefix + "sqce_std"])

    best_pearson -= best_pearson_std
    best_ll -= best_ll_std
    best_lml -= best_lml_std
    best_mse += best_mse_std
    best_qce += best_qce_std
    best_sqce = abs(best_sqce) + best_sqce_std

    for algo, name in algo_names:
        row = data[data["model"] == algo]
        print(f"    {name} & {num(row[prefix + 'pearson'], row[prefix + 'pearson_std'], best_pearson, 'max')} & {num(row[prefix + 'll'], row[prefix + 'll_std'], best_ll, 'max')} & {num(row[prefix + 'lml'], row[prefix + 'lml_std'], best_lml, 'max')} & {num(row[prefix + 'mse'], row[prefix + 'mse_std'], best_mse, 'min')} & {num(row[prefix + 'qce'], row[prefix + 'qce_std'], best_qce, 'min')} & {num(row[prefix + 'sqce'], row[prefix + 'sqce_std'], best_sqce, 'zero')} \\\\")
    print("\\end{tabular}")
create_table(data, "ood ")

\begin{tabular}{l|rrrrrr}
    \multicolumn{1}{l}{Model} & \multicolumn{1}{c}{Worst U/R Pearson} & \multicolumn{1}{c}{psLML} & \multicolumn{1}{c}{LML} & \multicolumn{1}{c}{MSE} & \multicolumn{1}{c}{QCE} & \multicolumn{1}{c}{sQCE} \\
    \hline
    MAP & $\bm{0.487 \pm 0.074}$ & $-11.945 \pm 2.042$ & $\bm{-11.945 \pm 2.042}$ & $\bm{0.267 \pm 0.041}$ & $0.382 \pm 0.012$ & $-0.382 \pm 0.012$ \\
    Deep Ensemble & $\bm{0.520 \pm 0.075}$ & $-6.126 \pm 1.422$ & $\bm{-12.113 \pm 2.074}$ & $\bm{0.249 \pm 0.043}$ & $0.283 \pm 0.027$ & $-0.283 \pm 0.027$ \\
    MCD & $\bm{0.491 \pm 0.079}$ & $-6.868 \pm 1.720$ & $\bm{-12.175 \pm 2.398}$ & $\bm{0.259 \pm 0.049}$ & $0.316 \pm 0.023$ & $-0.316 \pm 0.023$ \\
    MultiMCD & $\bm{0.516 \pm 0.078}$ & $-5.053 \pm 1.518$ & $\bm{-12.290 \pm 2.409}$ & $\bm{0.253 \pm 0.052}$ & $0.271 \pm 0.035$ & $-0.271 \pm 0.035$ \\
    SWAG & $\bm{0.473 \pm 0.078}$ & $-12.551 \pm 1.994$ & $\bm{-12.621 \pm 2.000}$ & $\bm{0.280 \pm 0.040}$ & $0.386 \pm 0.012$ & $-0.386 \pm